### Imports

In [30]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K

from imageio import mimsave
from IPython.display import display as display_fn
from IPython.display import Image, clear_output

#dependencies for the UI
import customtkinter as ctk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
from torchvision import transforms
import io

In [31]:
# Initialize global variables
content_image = None
style_image = None

def load_content_image():
    global content_image
    file_path = filedialog.askopenfilename(title="Select Content Image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    
    if file_path:
        content_image = Image.open(file_path)
        update_image_display(content_image, content_image_label)

def load_style_image():
    global style_image
    file_path = filedialog.askopenfilename(title="Select Style Image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    
    if file_path:
        style_image = Image.open(file_path)
        update_image_display(style_image, style_image_label)

def update_image_display(image, image_label):
    width, height = int(image.width / 2), int(image.height / 2)
    image = image.resize((width, height), Image.Resampling.LANCZOS)
    
    image_label.image = ImageTk.PhotoImage(image)
    image_label.configure(image=image_label.image)  # Updated to use 'configure'

def run_style_transfer():
    if content_image is None or style_image is None:
        result_label.config(text="Please upload both images.")
        return
    
    result_label.config(text="Processing...")
    
    # Assume neural_style_transfer(content_image, style_image) is defined
    output_image = neural_style_transfer(content_image, style_image)
    
    output_img_tk = ImageTk.PhotoImage(output_image)
    output_image_label.configure(image=output_img_tk)
    output_image_label.image = output_img_tk
    result_label.config(text="Transfer Complete!")


In [32]:
def run_style_transfer():
    # Load, visualize, and preprocess the images.

    def tensor_to_image(tensor):
        '''Converts a tensor to an image.'''
        tensor_shape = tf.shape(tensor)
        number_elem_shape = tf.shape(tensor_shape)
        if number_elem_shape > 3:
            assert tensor_shape[0] == 1
            tensor = tensor[0]
        return tf.keras.preprocessing.image.array_to_img(tensor)

    def load_img(path_to_img):
        '''Loads an image as a tensor and scales it to 512 pixels.'''
        max_dim = 512
        image = tf.io.read_file(path_to_img)
        image = tf.image.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)

        shape = tf.shape(image)[:-1]
        long_dim = max(shape)
        scale = max_dim / long_dim

        new_shape = tf.cast(shape * scale, tf.int32)
        image = tf.image.resize(image, new_shape)
        image = image[tf.newaxis, :]
        image = tf.image.convert_image_dtype(image, tf.uint8)

        return image

    def load_images(content_path, style_path):
        '''Loads the content and style images as tensors.'''
        content_image = load_img(content_path)
        style_image = load_img(style_path)

        return content_image, style_image

    def imshow(image, title=None):
        '''Displays an image with a corresponding title.'''
        if len(image.shape) > 3:
            image = tf.squeeze(image, axis=0)

        plt.imshow(image)
        if title:
            plt.title(title)

    def show_images_with_objects(images, titles=[]):
        '''Displays a row of images with corresponding titles.'''
        if len(images) != len(titles):
            return

        plt.figure(figsize=(20, 12))
        for idx, (image, title) in enumerate(zip(images, titles)):
            plt.subplot(1, len(images), idx + 1)
            plt.xticks([])
            plt.yticks([])
            imshow(image, title)

    def display_gif(gif_path):
        '''Displays the generated images as an animated gif.'''
        with open(gif_path, 'rb') as f:
            display(Image(data=f.read(), format='png'))

    def create_gif(gif_path, images):
        '''Creates an animation of generated images.'''
        mimsave(gif_path, images, fps=1)
        return gif_path

    def clip_image_values(image, min_value=0.0, max_value=255.0):
        '''Clips the image pixel values by the given min and max.'''
        return tf.clip_by_value(image, clip_value_min=min_value, clip_value_max=max_value)

    def preprocess_image(image):
        '''Centers the pixel values of a given image to use with VGG-19.'''
        image = tf.cast(image, dtype=tf.float32)
        image = tf.keras.applications.vgg19.preprocess_input(image)
        return image

    # --------------BUILDING THE MODEL

    K.clear_session()  # Clear session for consistent layer naming
    tmp_vgg = tf.keras.applications.vgg19.VGG19()  # Download VGG-19 model
    tmp_vgg.summary()  # Inspect layers
    del tmp_vgg  # Delete temporary variable

    # Specify layers of interest
    style_layers = [
        'block1_conv1', 
        'block2_conv1', 
        'block3_conv1', 
        'block4_conv1', 
        'block5_conv1'
    ]
    content_layers = ['block5_conv2']
    output_layers = style_layers + content_layers

    NUM_CONTENT_LAYERS = len(content_layers)
    NUM_STYLE_LAYERS = len(style_layers)

    def vgg_model(layer_names):
        '''Creates a VGG model that outputs selected content and style layers.'''
        vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False  # Freeze weights
        outputs = [vgg.get_layer(name).output for name in layer_names]
        return tf.keras.Model(inputs=vgg.input, outputs=outputs)

    K.clear_session()  # Clear session again
    vgg = vgg_model(output_layers)  # Create VGG-19 model
    vgg.summary()

    # --- Calculate style loss
    def get_style_loss(features, targets):
        return tf.reduce_mean(tf.square(features - targets))

    # --- Calculate content loss
    def get_content_loss(features, targets):
        return 0.5 * tf.reduce_sum(tf.square(features - targets))

    # --- Calculate the gram matrix
    def gram_matrix(input_tensor):
        gram = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        height, width = tf.shape(input_tensor)[1:3]
        num_locations = tf.cast(height * width, tf.float32)
        return gram / num_locations

    # --- Get the style image features
    def get_style_image_features(image):
        preprocessed_style_image = preprocess_image(image)
        outputs = vgg(preprocessed_style_image)
        style_outputs = outputs[:NUM_STYLE_LAYERS]
        return [gram_matrix(style_layer) for style_layer in style_outputs]

    # --- Get content image features
    def get_content_image_features(image):
        preprocessed_content_image = preprocess_image(image)
        outputs = vgg(preprocessed_content_image)
        return outputs[NUM_STYLE_LAYERS:]

    # --- Get style and content loss
    def get_style_content_loss(style_targets, style_outputs, content_targets, content_outputs, 
                               style_weight, content_weight):
        style_loss = tf.add_n([get_style_loss(style_output, style_target)
                                for style_output, style_target in zip(style_outputs, style_targets)])
        content_loss = tf.add_n([get_content_loss(content_output, content_target)
                                  for content_output, content_target in zip(content_outputs, content_targets)])
        style_loss *= style_weight / NUM_STYLE_LAYERS
        content_loss *= content_weight / NUM_CONTENT_LAYERS
        return style_loss + content_loss

    # ---------GENERATE THE STYLIZED IMAGE

    # Step 1 - calculate gradients
    def calculate_gradients(image, style_targets, content_targets, 
                            style_weight, content_weight):
        with tf.GradientTape() as tape:
            style_features = get_style_image_features(image)
            content_features = get_content_image_features(image)
            loss = get_style_content_loss(style_targets, style_features, content_targets, 
                                          content_features, style_weight, content_weight)
        return tape.gradient(loss, image)

    # Step 2 - update image with style
    def update_image_with_style(image, style_targets, content_targets, 
                                 style_weight, content_weight, optimizer):
        gradients = calculate_gradients(image, style_targets, content_targets, 
                                        style_weight, content_weight)
        optimizer.apply_gradients([(gradients, image)])
        image.assign(clip_image_values(image, min_value=0.0, max_value=255.0))

    def fit_style_transfer(style_image, content_image, style_weight=1e-2, 
                           content_weight=1e-4, optimizer='adam', epochs=1, steps_per_epoch=1):
        images = []
        step = 0

        # Ensure that these functions return valid tensors
        style_targets = get_style_image_features(style_image)  
        content_targets = get_content_image_features(content_image)

        generated_image = tf.Variable(tf.cast(content_image, dtype=tf.float32))
        images.append(content_image)

        for n in range(epochs):
            for m in range(steps_per_epoch):
                step += 1
                update_image_with_style(generated_image, style_targets, content_targets, 
                                        style_weight, content_weight, optimizer)
                print(".", end='')

                if (m + 1) % 10 == 0:
                    images.append(generated_image)

            clear_output(wait=True)
            display_image = tensor_to_image(generated_image)
            display(display_image)
            images.append(generated_image)
            print("Train step: {}".format(step))

        return tf.cast(generated_image, dtype=tf.uint8), images

    # Define style and content weight
    style_weight =  2e-2
    content_weight = 1e-2 

    # Define optimizer
    adam = tf.optimizers.Adam(
        tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=20.0, decay_steps=100, decay_rate=0.50
        )
    )

    # Start the neural style transfer
    stylized_image, display_images = fit_style_transfer(style_image=style_image, 
                                                        content_image=content_image, 
                                                        style_weight=style_weight, 
                                                        content_weight=content_weight,
                                                        optimizer=adam, epochs=10, steps_per_epoch=10)

    # Display GIF of Intermediate Outputs
    GIF_PATH = 'style_transfer.gif'
    gif_images = [np.squeeze(image.numpy().astype(np.uint8), axis=0) for image in display_images]
    gif_path = create_gif(GIF_PATH, gif_images)
    display_gif(gif_path)

# Call the function to run the style transfer
run_style_transfer()

Model: "vgg19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,31

 Total params: 143,667,240 (548.05 MB)

 Trainable params: 143,667,240 (548.05 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 15,304,768 (58.38 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,304,768 (58.38 MB)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:

# Initialize the main window
app = ctk.CTk()
app.geometry("1000x600")  # Adjusted size for better layout
app.title("Neural Style Transfer")

# Create content image section
content_label = ctk.CTkLabel(app, text="Content Image")
content_label.grid(row=0, column=0, padx=20, pady=20)
content_image_label = ctk.CTkLabel(app)
content_image_label.grid(row=1, column=0, padx=20, pady=20)
content_button = ctk.CTkButton(app, text="Upload Content Image", command=load_content_image)
content_button.grid(row=2, column=0, padx=20, pady=20)

# Create style image section
style_label = ctk.CTkLabel(app, text="Style Image")
style_label.grid(row=0, column=1, padx=20, pady=20)
style_image_label = ctk.CTkLabel(app)
style_image_label.grid(row=1, column=1, padx=20, pady=20)
style_button = ctk.CTkButton(app, text="Upload Style Image", command=load_style_image)
style_button.grid(row=2, column=1, padx=20, pady=20)

# Output image display section
output_label = ctk.CTkLabel(app, text="Output Image")
output_label.grid(row=0, column=2, padx=20, pady=20)
output_image_label = ctk.CTkLabel(app)
output_image_label.grid(row=1, column=2, padx=20, pady=20)

# Create result status label and transfer button
result_label = ctk.CTkLabel(app, text="")
result_label.grid(row=2, column=2, padx=20, pady=20)

transfer_button = ctk.CTkButton(app, text="Transfer Style", command=run_style_transfer)
transfer_button.grid(row=3, column=1, padx=20, pady=20)


# Start the main event loop
app.mainloop()


**Awesome work! You have now completed the labs for Neural Style Transfer!**